In [ ]:
'''
Define some helper functions (most of this code is adapted from cv_algorithms)
'''

import numpy as np
import pandas as pd
import statsmodels.api as sm
from PIL import Image, ImageDraw
import random
from scipy.optimize import linear_sum_assignment
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances


FOCAL_LENGTH = 0.0107
BASELINE = 0.135
PIXEL_SIZE_M = 3.45 * 1e-6
FOCAL_LENGTH_PIXEL = FOCAL_LENGTH / PIXEL_SIZE_M
IMAGE_SENSOR_WIDTH = 0.01412
IMAGE_SENSOR_HEIGHT = 0.01035
PIXEL_COUNT_WIDTH = 4096
PIXEL_COUNT_HEIGHT = 3000
COST_THRESHOLD = 100.0  # another magic number

# TODO (@Alok): this is hardcoded - not good, should come directly from coco file

WHOLE_FISH = 'salmon'
MAX_NUM_LICE = 5
MIN_LICE_WIDTH_PX, MAX_LICE_WIDTH_PX = 10, 30
LICE_CLASSES = ['not_lice', 'moving/fixed', 'adult female']


def convert_to_world_point(x, y, d):
    """ from pixel coordinates to world coordinates """
    
    image_center_x = PIXEL_COUNT_HEIGHT / 2.0  
    image_center_y = PIXEL_COUNT_WIDTH / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (IMAGE_SENSOR_WIDTH / 4096)
    sensor_z = px_z * (IMAGE_SENSOR_HEIGHT / 3000)

    # d = depth_map[y, x]
    world_y = d
    world_x = (world_y * sensor_x) / FOCAL_LENGTH
    world_z = (world_y * sensor_z) / FOCAL_LENGTH
    return np.array([world_x, world_y, world_z])


def depth_from_disp(disp):
    depth = FOCAL_LENGTH_PIXEL*BASELINE / np.array(disp)
    return depth


def get_lice_detections(full_fish_annotation):
    
    """ implementation of a 'dumb' lice counting algorithm """
    
    bbox = full_fish_annotation['bbox']
    num_lice = int(np.random.uniform(0, MAX_NUM_LICE))
    lice_detections = []
    for i in range(num_lice):
        # randomly generate a bounding box representing a lice detection
        centroid_x = np.random.unform(bbox[0] + MAX_LICE_WIDTH_PX, bbox[2] - MAX_LICE_WIDTH_PX)
        centroid_y = np.random.unform(bbox[1] + MAX_LICE_WIDTH_PX, bbox[3] - MAX_LICE_WIDTH_PX)
        lice_width = np.random.uniform(MIN_LICE_WIDTH_PX, MAX_LICE_WIDTH_PX)
        lice_height = np.random.uniform(MIN_LICE_WIDTH_PX, MAX_LICE_WIDTH_PX)
        bbox = (centroid_x - 0.5 * lice_width, centroid_y - 0.5 * lice_height, centroid_x + 0.5 * lice_width, centroid_y + 0.5 * lice_height)
        lice_class = random.sample(LICE_CLASSES, 1)[0]
        lice_detection = {
            'class': lice_class,
            'bbox': bbox
        }
        lice_detections.append(lice_detection)
    
    return lice_detections
        
